In [1]:
!pip install dlib

In [2]:
import os
import urllib.request
import shutil
import bz2

**Download the shape predictor model file from the dlib repository**

In [3]:
print("Downloading shape predictor model file...")
model_url = "http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2"
model_path_bz2 = "/content/shape_predictor_68_face_landmarks.dat.bz2"
model_path_dat = "/content/shape_predictor_68_face_landmarks.dat"
urllib.request.urlretrieve(model_url, model_path_bz2)

('/content/shape_predictor_68_face_landmarks.dat.bz2',
 <http.client.HTTPMessage at 0x7eff831a1f30>)

**Extract the downloaded file**

 Once the model file is downloaded as a compressed file (shape_predictor_68_face_landmarks.dat.bz2), the code extracts it to obtain the actual model file (shape_predictor_68_face_landmarks.dat). This step is necessary to ensure that the model file is in the correct format for dlib to load.

In [4]:
print("Extracting shape predictor model file...")
with bz2.open(model_path_bz2, "rb") as f_in:
    with open(model_path_dat, "wb") as f_out:
        shutil.copyfileobj(f_in, f_out)
os.remove(model_path_bz2)
print("Shape predictor model file downloaded and extracted successfully.")

Extracting shape predictor model file...
Shape predictor model file downloaded and extracted successfully.


In [5]:
import cv2
import dlib
import numpy as np
from matplotlib import pyplot as plt

**Load the pre-trained facial landmark detector**

In [6]:
detector = dlib.get_frontal_face_detector()
try:
    predictor = dlib.shape_predictor(model_path_dat)
except Exception as e:
    print("Error loading the shape predictor model:", e)
    exit()

**Function to detect eye landmarks**

In [7]:
def detect_eyes(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    for face in faces:
        landmarks = predictor(gray, face)
        left_eye = []
        right_eye = []
        for n in range(36, 42): # Left eye landmarks
            x = landmarks.part(n).x
            y = landmarks.part(n).y
            left_eye.append((x, y))
        for n in range(42, 48): # Right eye landmarks
            x = landmarks.part(n).x
            y = landmarks.part(n).y
            right_eye.append((x, y))
        return left_eye, right_eye
    return None, None

**Main function**

The program starts capturing video from the default camera (usually the webcam) using OpenCV's VideoCapture() function. It continuously processes each frame of the video stream.For each frame, the code detects faces using the dlib frontal face detector (get_frontal_face_detector()) and then uses the shape predictor model to detect landmarks for each detected face. Specifically, it detects the landmarks corresponding to the left and right eyes.

In [8]:
def main():
    cap = cv2.VideoCapture(0)
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        left_eye, right_eye = detect_eyes(frame)
        if left_eye and right_eye:
            # Draw left eye landmarks
            for (x, y) in left_eye:
                cv2.circle(frame, (x, y), 2, (0, 255, 0), -1)
            # Draw right eye landmarks
            for (x, y) in right_eye:
                cv2.circle(frame, (x, y), 2, (0, 255, 0), -1)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        plt.imshow(frame_rgb)
        plt.axis('off')
        plt.show()
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()

 the code successfully detects eye landmarks for a frame, it draws green circles around the detected landmarks for both eyes. It then displays the annotated frame using matplotlib's imshow() function within the notebook. This provides a visual representation of where the eyes are located in the frame.
User Interaction: While the video stream is being displayed, you can press the 'q' key on your keyboard to quit the program and close the video stream.



---



## **Test the model without camera**

**Load a static image**
Can be done either way:

*   Upload an image to /content/ directory of Google colab.
*   Enter a path to a sample image on your PC.





In [9]:
!wget https://github.com/meytiii/Deep-Learning/blob/main/datasets/sample-image.jpg?raw=true
#Remember to change the name to sample-image.jpg (Remove the "?raw=true" at the end)

--2024-05-03 10:09:12--  https://github.com/meytiii/Deep-Learning/blob/main/datasets/sample-image.jpg?raw=true
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/meytiii/Deep-Learning/raw/main/datasets/sample-image.jpg [following]
--2024-05-03 10:09:13--  https://github.com/meytiii/Deep-Learning/raw/main/datasets/sample-image.jpg
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/meytiii/Deep-Learning/main/datasets/sample-image.jpg [following]
--2024-05-03 10:09:13--  https://raw.githubusercontent.com/meytiii/Deep-Learning/main/datasets/sample-image.jpg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.

In [15]:
image_path = "/content/sample-image.jpg"
frame = cv2.imread(image_path)

**Perform eye landmark detection on the image**

In [16]:
left_eye, right_eye = detect_eyes(frame)

**Visualize the results (draw circles around the detected eye landmarks)**

In [17]:
if left_eye and right_eye:
    for (x, y) in left_eye:
        cv2.circle(frame, (x, y), 2, (0, 255, 0), -1)
    for (x, y) in right_eye:
        cv2.circle(frame, (x, y), 2, (0, 255, 0), -1)

**Display the annotated image**

In [ ]:
from google.colab.patches import cv2_imshow

cv2_imshow(frame)
cv2.waitKey(0)
cv2.destroyAllWindows()

**As you can see our model framed the person's eyes with green dots so we know that our model is working correctly.**

**Save the trained shape predictor model to a file using pickle**

In [19]:
import pickle

with open("eye_tracking_model.pkl", "wb") as f:
    pickle.dump(predictor, f)

**Load the saved shape predictor model from the file using pickle**


*   For later use...



In [20]:
with open("eye_tracking_model.pkl", "rb") as f:
    predictor = pickle.load(f)